In [1]:
# !pip install torchvision
# !pip install torchinfo
# !pip install torchmetrics
# !pip install wandb -qU

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, random_split
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import torchmetrics
from torchinfo import summary
import torchvision.models as models
%matplotlib inline

In [3]:
# importing the labels

labels_training = pd.read_csv("Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv")
labels_testing = pd.read_csv("Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv")
labels_training.dropna(axis = 1, inplace = True)
labels_training, labels_testing

(    Image name  Retinopathy grade  Risk of macular edema 
 0    IDRiD_001                  3                       2
 1    IDRiD_002                  3                       2
 2    IDRiD_003                  2                       2
 3    IDRiD_004                  3                       2
 4    IDRiD_005                  4                       0
 ..         ...                ...                     ...
 408  IDRiD_409                  2                       1
 409  IDRiD_410                  2                       0
 410  IDRiD_411                  2                       0
 411  IDRiD_412                  2                       0
 412  IDRiD_413                  2                       0
 
 [413 rows x 3 columns],
     Image name  Retinopathy grade  Risk of macular edema 
 0    IDRiD_001                  4                       0
 1    IDRiD_002                  4                       1
 2    IDRiD_003                  4                       0
 3    IDRiD_004              

In [4]:
enable_wandb =  True
use_gpu = True,
gpu_available = torch.cuda.is_available()

if enable_wandb:
  import wandb
  wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mwangcy0327 (bostonu-ba865). Use `wandb login --relogin` to force relogin


In [5]:
# defining the configurations

configs = {
    "experiment_name": "Modified",
    "freeze_feature_extraction_layers": False,
    "pretrained": True,
    "efficientnet": False, 
    "data_augmentation": True,
    "transferlearning": True,

    # Data
    "img_dimensions" : (3, 448, 600),
    "batch_size" : 32,
    "num_classes" : 5,

    # CNN
    "filter_sizes" : [4,8,16],
    "kernel_size" : 3,
    "stride": 1,
    "padding": 0,

    # Optimzation
    "learning_rate" : 0.0001,
    "epochs" : 10,
    "weight_decay" : 0.0001,
    "patience" : 5
}

In [6]:
training_path = "Disease Grading/1. Original Images/a. Training Set"
testing_path = "Disease Grading/1. Original Images/b. Testing Set"


# Defining transformations for training data with data augmentation
if configs["data_augmentation"]:
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.Resize([448, 500]),
        transforms.Pad((0, 10, 0, 10)),
        transforms.RandomRotation(45),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.5944, 0.2908, 0.0962], std=[0.2047, 0.1284, 0.0866])
    ])
    
# Defining transformations for training data without data augmentation
else:
    transform = transforms.Compose([
        transforms.Resize([448, 500]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5944, 0.2908, 0.0962], std=[0.2047, 0.1284, 0.0866])
    ])

# Define transformations for test data
test_transform = transforms.Compose([
    transforms.Resize(([448, 600])),
    transforms.Pad((0, 10, 0, 10)),
    transforms.ToTensor()
    ])

train_dataset = torchvision.datasets.ImageFolder(training_path, transform = transform)
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2])
test_dataset = torchvision.datasets.ImageFolder(testing_path, transform = test_transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size = configs["batch_size"], shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size = configs["batch_size"], shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size = configs["batch_size"], shuffle=False)

In [7]:
train_dataset[0][0].shape

torch.Size([3, 468, 500])

In [8]:
model = models.resnet18(pretrained=configs["pretrained"])

if gpu_available and use_gpu:
  model = model.cuda()

/usr4/ba820/mwangcy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr4/ba820/mwangcy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# configuring the last fully connected layer
if configs["transferlearning"]:
  if configs["efficientnet"]:
    model.classifier[1] = torch.nn.Sequential(torch.nn.Linear(1408, 100), torch.nn.ReLU(), torch.nn.Linear(100, configs["num_classes"]))
  else:
    model.fc = torch.nn.Sequential(torch.nn.Linear(512, 100), torch.nn.ReLU(), torch.nn.Linear(100, configs["num_classes"]))
model

if configs["transferlearning"]:
  if configs["freeze_feature_extraction_layers"]==True:
    # Freeze all layers except the last fully connected layer
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last fully connected layer
    if configs["efficientnet"]:
      for param in model.classifier[1].parameters():
          param.requires_grad = True
    else:
      for param in model.fc.parameters():
          param.requires_grad = True

summary(model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BasicBlock: 2-1                   --
│    │    └─Conv2d: 3-1                  36,864
│    │    └─BatchNorm2d: 3-2             128
│    │    └─ReLU: 3-3                    --
│    │    └─Conv2d: 3-4                  36,864
│    │    └─BatchNorm2d: 3-5             128
│    └─BasicBlock: 2-2                   --
│    │    └─Conv2d: 3-6                  36,864
│    │    └─BatchNorm2d: 3-7             128
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  36,864
│    │    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
│    └─BasicBlock: 2-3                   --
│    │    └─Conv2d: 3-11                 73,728

In [11]:
# defining loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=configs["learning_rate"], weight_decay=configs["weight_decay"])

In [12]:
def get_accuracy(dataloader, model):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  acc = torchmetrics.Accuracy(task = "multiclass", num_classes = configs["num_classes"]).to(device)

  with torch.no_grad():
      for images, labels in dataloader:
          if gpu_available and use_gpu:
            images = images.cuda()
            labels = labels.cuda()

          outputs = model(images) # get predictions

          # Update accuracy for this batch
          acc.update(outputs, labels)


      # Compute the accuracy
      acc = acc.compute()

      return acc

def get_loss(loader):
  with torch.no_grad(): 

    loss = 0
    for i, (images, labels) in enumerate(loader): # The batches.
          # step1: Move data to cuda. Make sure the model is on cuda too!
          if gpu_available and use_gpu:
            images = images.cuda()
            labels = labels.cuda()

          # step2: Forward pass
          outputs = model(images)

          # step 3: calculate the loss.
          loss = loss + criterion(outputs, labels)
    return loss/ len(loader)

class EarlyStopper:
    def __init__(self, patience=1):
        self.patience = patience
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        # If the new loss is lower than the old loss, reset the counter
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            # Keep track of the best model by saving it on the hard drive.
            torch.save(model.state_dict(), "./best_model.pt")
        # otherwise increase the counter.
        elif validation_loss > self.min_validation_loss:
            self.counter += 1
            # If there has been too more epochs than the defined patience, terminate.
            if self.counter >= self.patience:
                return True
        return False

In [13]:
if enable_wandb:
  wandb.init(
    project= "BA865 Project Neural Networks Report",
    name=configs["experiment_name"],
    config=configs)

if gpu_available and use_gpu:
  model = model.cuda()

In [14]:
early_stopper = EarlyStopper(patience = configs["patience"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.3, patience=3)

lowest_val_loss = float('inf')
best_model_weights = None

# Training loop
for epoch in range(configs["epochs"]): # The epochs.
    for i, (images, labels) in enumerate(train_loader): # The batches.
        # step 1: Zero out the gradients.
        optimizer.zero_grad()

        # step 1.1 move data to cuda. Make sure the model is on cuda too!
        if gpu_available and use_gpu:
          images = images.cuda()
          labels = labels.cuda()

        # step2: Forward pass
        outputs = model(images)

        # step 3: calculate the loss.
        loss = criterion(outputs, labels)

        # step 4: Backward pass
        loss.backward()
        optimizer.step()

        # Print the loss
        if i %10 == 0:
          print("Epoch", epoch+ 1, " batch", i+1, ". Training Loss: ", loss.item())
          if enable_wandb:
            wandb.log({"loss": loss})

    # Compute total train accuracy
    train_acc = get_accuracy(train_loader, model)
    test_acc = get_accuracy(val_loader, model)

    validation_loss = get_loss(val_loader)
    wandb.log({"val_loss": validation_loss})

    print(f'Epoch [{epoch + 1}/{configs["epochs"]}], LR: {optimizer.param_groups[0]["lr"]}, Validation Loss: {validation_loss.item():.4f}, Train Accuracy: {train_acc.item():.4f}, Validation Accuracy: {test_acc.item():.4f}')
    if enable_wandb:
      wandb.log({"epoch": epoch + 1, "LR": {optimizer.param_groups[0]['lr']}, "train_accuracy": train_acc.item(), "val_accuracy": test_acc.item()})

      predictions = []
    
      with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.cuda()) 
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())  # Append predictions to the list
    
      wandb.log({"conf_mat": wandb.plot.confusion_matrix(probs=None, 
                                                    y_true=test_loader.dataset.targets, 
                                                    preds=predictions, 
                                                    class_names=test_loader.dataset.classes)})

    scheduler.step(validation_loss)
    
    if validation_loss < lowest_val_loss:
        print("New lowest validation loss found. Saving model...")
        lowest_val_loss = validation_loss
        best_model_weights = model.state_dict()

    # Save the best model weights
    torch.save(best_model_weights, 'best_model_weights_no_norm.pth')
    
    scheduler.step(validation_loss)

    if early_stopper.early_stop(validation_loss):
        print("Validation loss hasn't dropped. Early stopping!")
        break

Epoch 1  batch 1 . Training Loss:  1.5394303798675537
Epoch 1  batch 11 . Training Loss:  1.128475308418274
Epoch [1/10], LR: 0.0001, Validation Loss: 1.1380, Train Accuracy: 0.5710, Validation Accuracy: 0.6098
New lowest validation loss found. Saving model...
Epoch 2  batch 1 . Training Loss:  1.2253285646438599
Epoch 2  batch 11 . Training Loss:  1.3086007833480835
Epoch [2/10], LR: 0.0001, Validation Loss: 0.9265, Train Accuracy: 0.6133, Validation Accuracy: 0.6707
New lowest validation loss found. Saving model...
Epoch 3  batch 1 . Training Loss:  1.091309905052185
Epoch 3  batch 11 . Training Loss:  0.9710489511489868
Epoch [3/10], LR: 0.0001, Validation Loss: 0.9390, Train Accuracy: 0.7039, Validation Accuracy: 0.6585
Epoch 4  batch 1 . Training Loss:  0.6959180235862732
Epoch 4  batch 11 . Training Loss:  0.5536024570465088
Epoch [4/10], LR: 0.0001, Validation Loss: 0.8285, Train Accuracy: 0.7009, Validation Accuracy: 0.6951
New lowest validation loss found. Saving model...
Epoc

In [15]:
if enable_wandb:
  wandb.finish()

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▆▇▅▅▃▂▃▃▃▄▂▂▂▄▁▁▁▂
train_accuracy,▁▂▄▄▆▆▇▇██
val_accuracy,▁▅▅▇▄█▄▃▅▄
val_loss,█▄▄▂▄▁▃▄▃▁
epoch,10
loss,0.56361
train_accuracy,0.85196
val_accuracy,0.64634
val_loss,0.80188
